In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
cleaned=pd.read_csv("C:/Users/avakk/Downloads/updatedcv.csv")

cleaned.drop(["url","lat","long","id","model","region"],axis=1,inplace=True)

cleaned.drop(cleaned[cleaned["title_status"]=="parts only"].index,inplace=True)
cleaned.drop(cleaned[cleaned["title_status"]=="missing"].index,inplace=True)
cleaned.drop(cleaned[cleaned["title_status"]=="lien"].index,inplace=True)

cleaned.drop(cleaned[cleaned["price"]<2000].index,inplace=True)
cleaned.drop(cleaned[cleaned["price"]>40000].index,inplace=True)

cleaned["price"]=np.log1p(cleaned["price"])

ne=["ct","me","ma","nh","ri","vt","nj","ny","pa"]
mw=["il","in","mi","oh","wi","ia","ks","mn","mo","ne","nd","sd"]
s=["de","fl","ga","md","nc","sc","va","wv","dc","ms","al","ky","tn","ar","tx","ok","la"]
w=["az","co","id","mt","nv","nm","ut","wy","wa","or","hi","ca","az","ak"]

for index, row in cleaned.iterrows():
    if(row["state"] in ne):
        cleaned.at[index,"state_trans"]="ne"
    if(row["state"] in mw):
        cleaned.at[index,"state_trans"]="mw"
    if(row["state"] in s):
        cleaned.at[index,"state_trans"]="s"
    if(row["state"] in w):
        cleaned.at[index,"state_trans"]="w"

yearmed=cleaned["year"].median()
cleaned["year"].fillna(yearmed)
for index, row in cleaned.iterrows():
    if(row["year"]>2020):
        cleaned.at[index,"year"]=yearmed

In [3]:
website_url = requests.get("https://en.wikipedia.org/wiki/List_of_U.S._states_by_GDP_per_capita").text
soup = BeautifulSoup(website_url,"lxml")

my_table = soup.find("table",{"class":"wikitable sortable"})
my_table
names=my_table.findAll('tr')
soup = BeautifulSoup(website_url,"html.parser")
rows = []
for tr in soup.select('tr'):
    rows.append([td.get_text(strip=True) for td in tr.select('th, td')])
    
rows_up=rows[:len(rows)-77]

gdp=pd.DataFrame(rows_up,columns=["Rank","State","2018","2017","2016","2015","2014","2013","2012","2011"])

gdp.drop(["Rank"],axis=1,inplace=True)
gdp.drop(gdp.index[[0,1,23]],inplace=True)
gdp["State"]=gdp["State"].str.lower()

gdp.columns=["US States","GDP2018","GDP2017","GDP2016","GDP2015","GDP2014","GDP2013","GDP2012","GDP2011"]
gdp.index=range(1,len(gdp)+1)

columns=["GDP2018","GDP2017","GDP2016","GDP2015","GDP2014","GDP2013","GDP2012","GDP2011"]

us_state_abbrev = {
    'alabama': 'AL',
    'alaska': 'AK',
    'american samoa': 'AS',
    'arizona': 'AZ',
    'arkansas': 'AR',
    'california': 'CA',
    'colorado': 'CO',
    'connecticut': 'CT',
    'delaware': 'DE',
    'district of columbia': 'DC',
    'florida': 'FL',
    'georgia': 'GA',
    'guam': 'GU',
    'hawaii': 'HI',
    'idaho': 'ID',
    'illinois': 'IL',
    'indiana': 'IN',
    'iowa': 'IA',
    'kansas': 'KS',
    'kentucky': 'KY',
    'louisiana': 'LA',
    'maine': 'ME',
    'maryland': 'MD',
    'massachusetts': 'MA',
    'michigan': 'MI',
    'minnesota': 'MN',
    'mississippi': 'MS',
    'missouri': 'MO',
    'montana': 'MT',
    'nebraska': 'NE',
    'nevada': 'NV',
    'new hampshire': 'NH',
    'new jersey': 'NJ',
    'new mexico': 'NM',
    'new york': 'NY',
    'north carolina': 'NC',
    'north dakota': 'ND',
    'northern mariana islands':'MP',
    'ohio': 'OH',
    'oklahoma': 'OK',
    'oregon': 'OR',
    'pennsylvania': 'PA',
    'puerto rico': 'PR',
    'rhode island': 'RI',
    'south carolina': 'SC',
    'south dakota': 'SD',
    'tennessee': 'TN',
    'texas': 'TX',
    'utah': 'UT',
    'vermont': 'VT',
    'virgin islands': 'VI',
    'virginia': 'VA',
    'washington': 'WA',
    'west virginia': 'WV',
    'wisconsin': 'WI',
    'wyoming': 'WY'
}

for index,row in gdp.iterrows():
    gdp.at[index,"US States"]=us_state_abbrev[row["US States"]]

gdp["US States"]=gdp["US States"].str.lower()
gdp.columns=["state","GDP2018","GDP2017","GDP2016","GDP2015","GDP2014","GDP2013","GDP2012","GDP2011"]


updatedsecondary=pd.merge(cleaned,gdp,on="state")

In [5]:
gdp

,state,GDP2018,GDP2017,GDP2016,GDP2015,GDP2014,GDP2013,GDP2012,GDP2011
1,ma,"65,545","64,507","62,510","61,882","62,456","61,769","60,808","59,178"
2,ny,"64,579","64,093","63,420","62,444","62,841","61,185","61,267","59,481"
3,ct,"64,511","63,747","62,236","62,550","63,502","63,638","64,906","65,574"
4,ak,"63,971","67,705","67,411","69,700","73,478","70,573","69,564","72,204"
5,de,"63,664","64,040","63,271","60,719","62,174","63,793","62,837","64,313"
6,nd,"62,837","67,305","71,056","67,651","68,105","57,066","52,185","48,858"
7,wy,"58,821","60,908","61,417","60,770","60,777","63,985","64,603","66,320"
8,ca,"58,619","57,328","55,374","53,855","52,974","52,099","51,871","51,733"
9,nj,"57,084","56,472","55,635","55,750","55,161","54,302","55,023","55,067"
10,wa,"56,831","55,780","55,001","54,161","53,439","52,500","52,679","52,264"


In [6]:
updatedsecondary

,price,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,...,state,state_trans,GDP2018,GDP2017,GDP2016,GDP2015,GDP2014,GDP2013,GDP2012,GDP2011
0,9.792556,2012.0,volkswagen,excellent,4 cylinders,gas,63500.0,clean,manual,4wd,...,ut,w,"44,636","44,204","43,069","42,267","41,929","41,992","41,500","41,563"
1,9.546813,2009.0,ram,good,6 cylinders,gas,120057.0,clean,automatic,4wd,...,ut,w,"44,636","44,204","43,069","42,267","41,929","41,992","41,500","41,563"
2,10.448744,2017.0,ford,excellent,6 cylinders,diesel,95484.0,clean,automatic,4wd,...,ut,w,"44,636","44,204","43,069","42,267","41,929","41,992","41,500","41,563"
3,9.581973,2007.0,gmc,excellent,6 cylinders,gas,89340.0,clean,automatic,4wd,...,ut,w,"44,636","44,204","43,069","42,267","41,929","41,992","41,500","41,563"
4,9.433564,2013.0,ford,fair,6 cylinders,gas,226031.0,clean,automatic,4wd,...,ut,w,"44,636","44,204","43,069","42,267","41,929","41,992","41,500","41,563"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416739,9.770013,2015.0,ford,good,6 cylinders,gas,67634.0,clean,automatic,2wd,...,tn,s,"43,267","42,797","41,828","41,487","41,091","40,193","39,370","39,096"
416740,9.539716,2015.0,kia,good,6 cylinders,gas,25623.0,clean,automatic,2wd,...,tn,s,"43,267","42,797","41,828","41,487","41,091","40,193","39,370","39,096"
416741,10.225607,2016.0,audi,good,6 cylinders,gas,29344.0,clean,automatic,4wd,...,tn,s,"43,267","42,797","41,828","41,487","41,091","40,193","39,370","39,096"
416742,9.769956,2018.0,jeep,like new,6 cylinders,gas,52449.0,clean,automatic,4wd,...,tn,s,"43,267","42,797","41,828","41,487","41,091","40,193","39,370","39,096"


In [ ]:
updatedsecondary.to_csv(r'C:/Users/avakk/Downloads/primarysecondary.csv',index=False)